In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn import ensemble, model_selection, metrics 
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn import model_selection, linear_model, metrics, pipeline, preprocessing

In [2]:
data = pd.read_csv('contest_train_binary.csv')
data.head()

,ID,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,1053371037,1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,...,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,76507184,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,...,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,76753690,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,...,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,1026167358,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,...,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,7220095746,0,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,...,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


In [3]:
X_train = data[data.columns[2:]]
y_train = data.TARGET
X_train.head()

,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,-17.218055,0.0,...,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,NaN,3.0,...,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,-6.680747,0.0,...,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,36.659534,0.0,...,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,NaN,3.0,...,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


In [4]:
def mad_based_outlier(points, thresh=3.5): # median-absolute-deviation test
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum(abs(points - median), axis=1)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh
# отбросим аномалии
res = mad_based_outlier(X_train)
X_train = X_train[res]
y_train = y_train[res]

In [5]:
# colours = ['#000099', '#ffff00']
# plt.figure(figsize=(12,12))
# sns.heatmap(data[data.columns[72:75]].isnull(), cmap=sns.color_palette(colours))
missings = [] # find missing persentage in columns
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    missings.append((col, pct_missing*100))
#     print('{} - {}%'.format(col, round(pct_missing*100)))
missings.sort(key=lambda item: item[1])
# отбросим 7 штук с самыми большими процентами
columns_to_fill_999 = []
for item in missings[-7:]:
    columns_to_fill_999.append(item[0])
for column in columns_to_fill_999:
    X_train.loc[:, column].fillna(-999, inplace=True)

In [6]:
cat_columns = [] #далее будем работать с каждым из типов признаков отдельно
real_columns = []
for column in X_train.columns:
    if len(data[column].value_counts()) < 6:
        cat_columns.append(column)
    else:
        real_columns.append(column)
#         print(data[column].value_counts())
X_train_cat = X_train[cat_columns]
X_train_real = X_train[real_columns]
print('number of cat features:', len(cat_columns))
print('number of numerical features:', len(real_columns))

number of cat features: 32
number of numerical features: 228


In [7]:
# займемся обработкой пропущенных значений в численных признаках
means = []
for col in real_columns:
    missing = X_train_real[col].isnull()
    if missing.any():  # only do the imputation for the columns that have missing values.
#         print('imputing missing values for: {}'.format(col))
#         X_train_real['{}_ismissing'.format(col)] = missing
        mean = X_train_real[col].mean()
        means.append(mean)
        X_train_real.loc[:, col].fillna(mean, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
# займемся обработкой пропущенных значений в категориальных признаках
tops = []
for col in cat_columns:
    missings = X_train_cat[col].isnull()
    
    if missings.any() > 0:  # only do the imputation for the columns that have missing values.
#         print('imputing missing values for: {}'.format(col))
#         X_train_cat['{}_ismissing'.format(col)] = missing
        top = X_train_cat[col].value_counts().argmax() # impute with the most frequent value.
        tops.append(top)
        X_train_cat.loc[:, col].fillna(top, inplace=True)

In [9]:
X_train = np.hstack((X_train_real, X_train_cat))

In [10]:
def plot_correlation_matrix(corr_mtrx):
    f = plt.figure(figsize=(20, 20))
    plt.matshow(corr_mtrx, fignum=f.number)
    plt.xticks(range(corr_mtrx.shape[1]), fontsize=9, rotation=45)
    plt.yticks(range(corr_mtrx.shape[1]), fontsize=9)
    cb = plt.colorbar()
    plt.title('Correlation Matrix', fontsize=10)
def correlation_matrix(x):
    C = np.corrcoef(x.T)
#     # YOUR CODE HERE
#     x = x[:, :-1]
#     n = x.shape[1]
#     C = np.zeros((n, n))
#     for i in range(n):
#         for j in range(n):
#             C[i][j] = stats.pearsonr(x[:, i], x[:, j])[0]
    return C

corr_matrix = correlation_matrix(X_train)
# plot_correlation_matrix(corr_matrix)
correlated_columns_to_drop = set()
for i in range(corr_matrix.shape[0]):
    for j in range(i):
        if corr_matrix[i][j] > 0.8:
            correlated_columns_to_drop.add(i)
len(correlated_columns_to_drop)
X_train = pd.DataFrame(X_train).drop(columns=correlated_columns_to_drop)
print(X_train.shape)
X_train.head()

(3656, 191)


/Users/andrewbelyaev/Library/Python/3.7/lib/python/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/andrewbelyaev/Library/Python/3.7/lib/python/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,0,1,2,3,4,5,6,7,8,9,...,247,248,251,252,253,255,256,257,258,259
0,1.0,1.0,329.828334,587.391298,3.0,2.0,767.317882,30.0,11.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1,1.0,1.0,1059.073032,587.391298,3.0,2.0,1856.174543,15.0,6.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
2,14.0,3.0,905.326604,-34.180189,1.0,2.0,1177.823360,36.0,6.0,0.0,...,1.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,1.0
3,2.0,1.0,15.040910,6772.531073,2.0,1.0,2993.564139,121.0,16.0,0.0,...,1.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,2.0,2.0
4,19.0,3.0,849.270605,-10.653991,1.0,2.0,752.643438,24.0,7.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0


In [11]:
estimator = xgb.XGBClassifier(n_estimators=1000, max_depth=10, n_jobs=-1)
estimator.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
X_test = pd.read_csv('contest_test.csv')
X_test.head()
ID = X_test.ID
for column in columns_to_fill_999:
    X_test.loc[:, column].fillna(-999, inplace=True)

In [13]:
X_test_cat = X_test[cat_columns]
X_test_real = X_test[real_columns]
# займемся обработкой пропущенных значений в численных признаках
means = []
for col in real_columns:
    missing = X_test_real[col].isnull()
    if missing.any():  # only do the imputation for the columns that have missing values.
        mean = X_test_real[col].mean()
        X_test_real.loc[:, col].fillna(mean, inplace=True)
# займемся обработкой пропущенных значений в категориальных признаках
for col in cat_columns:
    missings = X_test_cat[col].isnull()
    
    if missings.any() > 0:  # only do the imputation for the columns that have missing values.
        top = X_test_cat[col].value_counts().argmax() # impute with the most frequent value.
        X_test_cat.loc[:, col].fillna(top, inplace=True)
X_test = np.hstack((X_test_real, X_test_cat))
X_test = pd.DataFrame(X_test).drop(columns=list(correlated_columns_to_drop))
predictions = estimator.predict_proba(X_test)
with open('submission2.csv', 'w') as f:
    f.write('ID,TARGET\n')
    for item in zip(ID, predictions):
        f.write(str(item[0]) + ',' + str(item[1][1]) + '\n')

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
